# (b) Training Results with 3 Hidden Neurons

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.backend as K

# List of seeds to try
SEEDS = [0, 1, 42]

# Configuration dictionary
config = {
    'batch_size': 10,
    'epochs': 500,
    'patience': 30,
    'hidden_neurons': 3  # Changed from 1 to 3
}

# Predefined trials as per assignment requirements
trials = {
    'A': {'learning_rate': 0.1, 'momentum': 0.1},
    'B': {'learning_rate': 0.1, 'momentum': 0.9},
    'C': {'learning_rate': 0.5, 'momentum': 0.5},
    'D': {'learning_rate': 0.9, 'momentum': 0.1},
    'E': {'learning_rate': 0.9, 'momentum': 0.9}
}

def load_and_preprocess_data(seed):
    """Load and preprocess the data with the given seed"""
    # Load data
    file_path = 'Heat_Influx_insulation_east_south_north.csv'
    data = pd.read_csv(file_path)
    
    # Define features and target
    features = ['Insulation', 'East', 'South', 'North']
    target = 'HeatFlux'
    
    # Normalize the data
    scaler = MinMaxScaler()
    data_normalized = pd.DataFrame(
        scaler.fit_transform(data[features + [target]]),
        columns=features + [target]
    )
    
    # Split the dataset (60% train, 20% validation, 20% test)
    train_data, temp_data = train_test_split(data_normalized, train_size=0.6, random_state=seed)
    val_data, test_data = train_test_split(temp_data, train_size=0.5, random_state=seed)
    
    # Prepare data sets
    X_train = train_data[features].values
    y_train = train_data[target].values
    X_val = val_data[features].values
    y_val = val_data[target].values
    X_test = test_data[features].values
    y_test = test_data[target].values
    X_all = data_normalized[features].values
    y_all = data_normalized[target].values
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_all, y_all

def build_and_train_model(X_train, y_train, X_val, y_val, learning_rate, momentum, seed):
    """Build and train the model with given parameters"""
    K.clear_session()
    tf.random.set_seed(seed)
    
    # Define the model
    initializer = tf.keras.initializers.GlorotUniform(seed=seed)
    model = Sequential([
        Input(shape=(4,)),
        Dense(3, activation='sigmoid', kernel_initializer=initializer),  # Changed from 1 to 3
        Dense(1, activation='linear', kernel_initializer=initializer)
    ])
    
    optimizer = SGD(learning_rate=learning_rate, momentum=momentum)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    
    # Early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=config['patience'],
        restore_best_weights=True
    )
    
    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=config['epochs'],
        batch_size=config['batch_size'],
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=0
    )
    
    return model, history

def evaluate_model(model, X_train, y_train, X_val, y_val, X_test, y_test, X_all, y_all):
    """Evaluate the model and return metrics"""
    y_train_pred = model.predict(X_train).flatten()
    y_val_pred = model.predict(X_val).flatten()
    y_test_pred = model.predict(X_test).flatten()
    y_all_pred = model.predict(X_all).flatten()
    
    metrics = {
        'MSE_Trn': mean_squared_error(y_train, y_train_pred),
        'MSE_Val': mean_squared_error(y_val, y_val_pred),
        'MSE_Test': mean_squared_error(y_test, y_test_pred),
        'MSE_All': mean_squared_error(y_all, y_all_pred),
        'R2_Trn': r2_score(y_train, y_train_pred),
        'R2_Val': r2_score(y_val, y_val_pred),
        'R2_Test': r2_score(y_test, y_test_pred),
        'R2_All': r2_score(y_all, y_all_pred)
    }
    
    return metrics

# Initialize results dictionary
results = {
    'Trial': [], 'Seed': [], 'Learning Rate': [], 'Momentum': [],
    'MSE_Trn': [], 'MSE_Val': [], 'MSE_Test': [], 'MSE_All': [],
    'R2_Trn': [], 'R2_Val': [], 'R2_Test': [], 'R2_All': []
}

# Train models for each trial and seed combination
for trial, params in trials.items():
    best_test_mse = float('inf')
    best_seed = None
    best_metrics = None
    
    print(f"\nTraining Trial {trial}: Learning Rate = {params['learning_rate']}, Momentum = {params['momentum']}")
    
    for seed in SEEDS:
        print(f"  Testing seed {seed}")
        
        # Load and preprocess data with current seed
        X_train, y_train, X_val, y_val, X_test, y_test, X_all, y_all = load_and_preprocess_data(seed)
        
        # Train model
        model, history = build_and_train_model(
            X_train, y_train, X_val, y_val,
            params['learning_rate'], params['momentum'], seed
        )
        
        # Evaluate model
        metrics = evaluate_model(model, X_train, y_train, X_val, y_val, X_test, y_test, X_all, y_all)
        
        # Check if this seed produced better results
        if metrics['MSE_Test'] < best_test_mse:
            best_test_mse = metrics['MSE_Test']
            best_seed = seed
            best_metrics = metrics
    
    # Store results for best seed
    results['Trial'].append(trial)
    results['Seed'].append(best_seed)
    results['Learning Rate'].append(params['learning_rate'])
    results['Momentum'].append(params['momentum'])
    for key, value in best_metrics.items():
        results[key].append(value)
    
    print(f"  Best seed: {best_seed}, Test MSE: {best_test_mse:.6f}")

# Create and display results DataFrame
results_df = pd.DataFrame(results)
print("\nResults Table:")
print(results_df.to_string(index=False))

# Save results to CSV
results_df.to_csv('ffnn_trials_results.csv', index=False)

# Print best model based on test MSE
best_test_mse = results_df.loc[results_df['MSE_Test'].idxmin()]
print("\nBest Model Based on Test Set MSE:")
print(f"Trial: {best_test_mse['Trial']}")
print(f"Seed: {best_test_mse['Seed']}")
print(f"Learning Rate: {best_test_mse['Learning Rate']}")
print(f"Momentum: {best_test_mse['Momentum']}")
print(f"Test MSE: {best_test_mse['MSE_Test']:.6f}")
print(f"Test R²: {best_test_mse['R2_Test']:.6f}")


Training Trial A: Learning Rate = 0.1, Momentum = 0.1
  Testing seed 0

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
  Testing seed 1
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
  Testing seed 42
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
  Best seed: 42, Test MSE: 0.004072

Training Trial B: Learning Rate = 0.1, Momentum = 0.9
  Testing seed 0
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 281us/step
  Testing seed 1
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 484u